# Documenting Your Code

## Why Documentation Matters

Documentation is one of the most important yet often neglected aspects of software development. For scientific computing and statistical methods, good documentation serves several critical purposes:

* *Reproducibility:* Well-documented code enables others to understand, verify, and reproduce your analyses. This is fundamental to scientific research.

* *Collaboration:* When working with others (or your future self), documentation explains not just *what* the code does, but *why* it does it that way.

* *Maintainability:* Code without documentation becomes difficult to modify or debug, even shortly after writing it.

* *Adoption:* If you develop a statistical method, clear documentation dramatically increases the likelihood that others will use it.

Think about who will read your documentation:

1. **Future you** - In six months, you won't remember why you made certain choices
2. **Collaborators** - Lab members, co-authors, or reviewers examining your analysis
3. **Users** - People who want to use your functions or packages

In this lecture, we'll cover the main documentation tools in Python: comments, docstrings, and type hints.

## Comments

Comments are explanatory text in your code that Python ignores during execution. They begin with the `#` symbol.

In [ ]:
# This is a single-line comment
x = 10

You can also place comments at the end of a line of code. These are called inline comments.

In [ ]:
x = 10  # This is an inline comment

Inline comments should be separated from the code by at least two spaces.

### When to Use Comments

The key principle is that comments should explain *why* something is done, not *what* is being done. The code itself should be clear enough to show what it does. Consider this example of a good comment that explains the reasoning:

In [ ]:
# Use log transform to stabilize variance for count data
log_counts = np.log1p(counts)

This comment adds value because it explains the statistical motivation, not the mechanics. Contrast this with a bad comment that merely restates what the code does:

In [ ]:
# Add 1 to x
x = x + 1

This comment is redundant because any Python programmer can see that `x + 1` adds 1 to x.

### Block Comments

Block comments appear on their own line(s) before the code they describe. They're useful for longer explanations that wouldn't fit on a single line.

In [ ]:
# Calculate the sample variance using Bessel's correction (n-1)
# to get an unbiased estimate of the population variance
variance = np.sum((x - mean) ** 2) / (n - 1)

This block comment explains both what statistical quantity is being computed and why the denominator is `n-1` rather than `n`.

### Inline Comments

Inline comments appear on the same line as code. Use them sparingly and only for brief clarifications.

In [ ]:
max_iter = 1000  # Convergence usually occurs within 100 iterations

This inline comment explains why the value 1000 was chosen, which isn't obvious from the code alone.

### Question

Consider the following function. Identify which comments add value and which should be removed or rewritten.

In [ ]:
def calculate_correlation(x, y):
    # Calculate mean of x
    mean_x = np.mean(x)
    # Calculate mean of y
    mean_y = np.mean(y)

    # Subtract means
    x_centered = x - mean_x
    y_centered = y - mean_y

    # Pearson correlation is undefined for constant arrays
    if np.std(x) == 0 or np.std(y) == 0:
        return np.nan

    # Calculate correlation
    numerator = np.sum(x_centered * y_centered)
    denominator = np.sqrt(np.sum(x_centered**2) * np.sum(y_centered**2))
    r = numerator / denominator  # Divide numerator by denominator

    return r

**Answer:**

The comments `# Calculate mean of x`, `# Calculate mean of y`, and `# Divide numerator by denominator` are all redundant. They simply restate what the code clearly shows.

The comment `# Pearson correlation is undefined for constant arrays` is valuable because it explains *why* we check for zero standard deviation - this is domain knowledge that isn't obvious from the code.

Improved version:

## Docstrings

Docstrings are string literals that appear as the first statement in a module, function, class, or method. Unlike comments, docstrings become part of the object and can be accessed programmatically.

### PEP 257 Conventions

[PEP 257](https://peps.python.org/pep-0257/) defines the standard conventions for docstrings. The main rules are:

* Use triple double quotes `"""` for all docstrings
* The first line should be a brief summary ending with a period
* If more detail is needed, leave a blank line after the summary

### One-Line Docstrings

For simple functions where the behavior is obvious, a single line is sufficient.

In [ ]:
def square(x):
    """Return the square of x."""
    return x ** 2

Notice three important conventions here. First, the opening and closing quotes are on the same line as the text. Second, the docstring is phrased as a command ("Return...") rather than a description ("Returns..."). Third, it ends with a period.

Here's another example of a well-written one-line docstring:

In [ ]:
def is_positive(n):
    """Check whether n is a positive number."""
    return n > 0

The docstring describes what the function checks, which is more informative than just saying "Return True if n > 0."

### Multi-Line Docstrings

For more complex functions, use multiple lines to provide additional detail.

In [ ]:
def fit_linear_model(X, y):
    """Fit an ordinary least squares linear regression model.

    Uses the normal equations to find the coefficients that minimize
    the sum of squared residuals.
    """
    # implementation

The structure is: summary line, blank line, then elaboration. The summary line should still be a complete sentence that stands on its own.

### Accessing Docstrings

One of the key benefits of docstrings over comments is that they're accessible at runtime. Every object with a docstring stores it in the `__doc__` attribute.

In [ ]:
def greet(name):
    """Return a greeting message for the given name."""
    return f"Hello, {name}!"

You can access this docstring directly:

In [ ]:
print(greet.__doc__)

This outputs: `Return a greeting message for the given name.`

The built-in `help()` function provides a more formatted view:

In [ ]:
help(greet)

This outputs:

```
Help on function greet in module __main__:

greet(name)
    Return a greeting message for the given name.
```

This is how users discover how to use your functions in an interactive Python session.

### Module Docstrings

Modules should have a docstring at the very top of the file, before any imports.

In [ ]:
"""Statistical utility functions for biomarker analysis.

This module provides functions for calculating summary statistics,
performing normalization, and handling missing data in biomarker datasets.
"""

import numpy as np

When users import your module and call `help(module_name)`, they'll see this docstring.

### Class Docstrings

Classes should document their purpose and list any public attributes.

In [ ]:
class LinearRegression:
    """Ordinary least squares linear regression.

    Fits a linear model to minimize the residual sum of squares
    between observed and predicted values.

    Attributes:
        coef_: Estimated coefficients after fitting.
        intercept_: Estimated intercept after fitting.
    """

    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

The class docstring describes what instances represent. Individual methods like `fit()` and `predict()` would have their own docstrings describing their specific behavior.

### Question

The following function is missing a docstring. Three candidates are provided below. Which is the best choice, and what's wrong with the others?

In [ ]:
def winsorize(data, limits=(0.05, 0.05)):
    lower_limit, upper_limit = limits
    lower_percentile = np.percentile(data, lower_limit * 100)
    upper_percentile = np.percentile(data, (1 - upper_limit) * 100)
    return np.clip(data, lower_percentile, upper_percentile)

**Option A:**

In [ ]:
"""Winsorize data."""

**Option B:**

In [ ]:
"""Winsorize data by clipping extreme values.

This function takes a data array and clips values that fall below
the lower percentile or above the upper percentile, replacing them
with the percentile values themselves.

Args:
    data: The input data array to be winsorized.
    limits: A tuple specifying the fraction of data to clip.
"""

**Option C:**

In [ ]:
"""Clip extreme values to specified percentiles.

Args:
    data: Input array of numeric values.
    limits: Tuple of (lower, upper) fractions. Values below the
        lower percentile and above the (1-upper) percentile are
        clipped. Defaults to (0.05, 0.05) for 5% on each tail.

Returns:
    Array with extreme values replaced by percentile boundaries.

Example:
    >>> winsorize([1, 2, 3, 100], limits=(0.25, 0.25))
    array([1.75, 2., 3., 3.])
"""

**Answer: Option C is best.**

* **Option A** is too brief. Users wouldn't know what "winsorize" means or how to use the function.
* **Option B** is better but has problems: it doesn't document the return value, doesn't explain the default, and the description of `limits` is vague about which percentile corresponds to which limit.
* **Option C** clearly documents the return value, explains the asymmetric percentile calculation `(1-upper)`, provides the default value, and includes an example showing the actual behavior.

## Docstring Formats

There are several standard formats for structuring docstrings. We'll focus on Google style, which is clean and widely used.

### Google Style

The [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html) defines a readable format with clear section headers.

In [ ]:
def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for the mean.

    Computes a confidence interval assuming the data comes from
    a normal distribution, using the t-distribution for small samples.

    Args:
        data: Array of numeric values.
        confidence: Confidence level between 0 and 1. Defaults to 0.95.

    Returns:
        A tuple of (lower_bound, upper_bound) for the confidence interval.

    Raises:
        ValueError: If confidence is not between 0 and 1.
        ValueError: If data has fewer than 2 observations.

    Example:
        >>> calculate_confidence_interval([1, 2, 3, 4, 5])
        (1.036, 4.964)
    """

The key sections are `Args` for parameters, `Returns` for the return value, `Raises` for exceptions, and optionally `Example` for usage demonstrations.

Each parameter in the `Args` section starts with the parameter name, followed by a colon, then a description. If the parameter has a default value, mention it in the description.

The `Returns` section describes both the type and the meaning of the return value. For tuples or complex objects, explain what each component represents.

The `Raises` section lists exceptions that the function might raise and the conditions that trigger them. This helps users understand what error handling they might need.

### NumPy Style

NumPy style is popular in scientific Python libraries (NumPy, SciPy, pandas). It uses section headers with underlines.

In [ ]:
def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for the mean.

    Parameters
    ----------
    data : array_like
        Array of numeric values.
    confidence : float, optional
        Confidence level between 0 and 1. Default is 0.95.

    Returns
    -------
    tuple
        Lower and upper bounds of the confidence interval.
    """

Each section name (`Parameters`, `Returns`, `Raises`) is followed by a line of dashes. Parameter entries include the type after a colon.

This format is more verbose than Google style but renders nicely in documentation tools. Use it when contributing to projects that already follow this convention.

### Question

The following function has several documentation errors. Identify all the problems.

In [ ]:
def robust_scale(data, center="median", scale="iqr", iqr_range=(25, 75)):
    """Scale data using robust statistics.

    Args:
        data: Input array.
        center: Method for centering ("median" or "mean").
        scale: Method for scaling.

    Returns:
        Scaled data array.
    """
    if center == "median":
        loc = np.median(data)
    elif center == "mean":
        loc = np.mean(data)
    else:
        raise ValueError(f"Unknown center method: {center}")

    if scale == "iqr":
        q1, q3 = np.percentile(data, iqr_range)
        sc = q3 - q1
    elif scale == "std":
        sc = np.std(data)
    else:
        raise ValueError(f"Unknown scale method: {scale}")

    if sc == 0:
        return np.zeros_like(data)

    return (data - loc) / sc

**Answer:**

1. **Missing parameter:** `iqr_range` is not documented in Args.
2. **Incomplete `scale` description:** The docstring says "Method for scaling" but doesn't list the valid options ("iqr" or "std").
3. **Missing default values:** None of the defaults (center="median", scale="iqr", iqr_range=(25, 75)) are mentioned.
4. **Missing Raises section:** The function raises `ValueError` for invalid center/scale methods, but this isn't documented.
5. **Missing edge case:** When `sc == 0`, the function returns zeros instead of the scaled data, but this special behavior isn't documented.

Corrected docstring:

In [ ]:
def robust_scale(data, center="median", scale="iqr", iqr_range=(25, 75)):
    """Scale data using robust statistics.

    Args:
        data: Input array of numeric values.
        center: Method for centering, either "median" or "mean".
            Defaults to "median".
        scale: Method for scaling, either "iqr" (interquartile range)
            or "std" (standard deviation). Defaults to "iqr".
        iqr_range: Percentiles for IQR calculation. Defaults to (25, 75).

    Returns:
        Scaled array with center removed and divided by scale.
        Returns zeros if scale is zero (constant data).

    Raises:
        ValueError: If center is not "median" or "mean".
        ValueError: If scale is not "iqr" or "std".
    """

## Type Hints

Type hints (introduced in [PEP 484](https://docs.python.org/3/library/typing.html)) allow you to specify the expected types of variables, parameters, and return values. They don't affect runtime behavior but improve code clarity and enable static analysis tools.

### Basic Syntax

The syntax uses a colon after parameter names and `->` before the return type.

In [ ]:
def greet(name: str) -> str:
    return f"Hello, {name}!"

Here `name: str` indicates the parameter should be a string, and `-> str` indicates the function returns a string.

For functions with multiple parameters, each one is annotated separately.

In [ ]:
def add(a: int, b: int) -> int:
    return a + b

This makes the function's contract clear: it takes two integers and returns an integer.

### Common Basic Types

The common types you'll use most often are `int`, `float`, `str`, `bool`, `list`, `dict`, `tuple`, and `set`.

In [ ]:
def calculate_mean(values: list) -> float:
    return sum(values) / len(values)

This says the function takes a list and returns a float. However, we can be more specific about what the list contains.

### Collection Types with Element Types

Use bracket notation to specify what types a collection contains.

In [ ]:
def sum_values(numbers: list[float]) -> float:
    return sum(numbers)

Here `list[float]` means a list containing float values. This is more informative than just `list`.

For dictionaries, specify both key and value types.

In [ ]:
def count_words(text: str) -> dict[str, int]:
    words = text.lower().split()
    counts = {}
    for word in words:
        counts[word] = counts.get(word, 0) + 1
    return counts

The return type `dict[str, int]` indicates a dictionary with string keys and integer values.

### Optional and Union Types

Use `|` (Python 3.10+) for values that can be one of several types.

In [ ]:
def process_input(value: int | float) -> float:
    """Accept either int or float, return float."""
    return float(value) * 2

This explicitly states that the function accepts both integers and floating-point numbers.

A very common pattern is functions that might return `None`.

In [ ]:
def find_index(items: list, target: str) -> int | None:
    """Return index of target, or None if not found."""
    try:
        return items.index(target)
    except ValueError:
        return None

The return type `int | None` indicates the function returns either an integer or None. This makes it clear to callers that they need to handle the None case.

For parameters that default to None, use the same syntax.

In [ ]:
def load_data(path: str, sep: str | None = None) -> list:
    """Load data from file. If sep is None, auto-detect delimiter."""
    # implementation

This shows that `sep` can be explicitly passed as a string or left as None for auto-detection.

### Question

Consider the following function with type hints. A colleague argues that the type hints are incorrect or incomplete. Identify what issues exist and explain how you would fix them.

In [ ]:
def safe_divide(a: float, b: float, default: float = None) -> float:
    """Divide a by b, returning default if b is zero."""
    if b == 0:
        return default
    return a / b

**Answer:**

There are two issues with these type hints:

1. **The `default` parameter type is wrong.** The hint says `float`, but the default value is `None`. This should be `float | None = None`.

2. **The return type is incomplete.** The function can return `None` when `b == 0` and `default` is `None`. The return type should be `float | None`, not just `float`.

Corrected signature:

In [ ]:
def safe_divide(a: float, b: float, default: float | None = None) -> float | None:

### Combining Type Hints with Docstrings

When you use type hints, docstrings can focus on meaning rather than types.

In [ ]:
def fit_model(
    X: np.ndarray,
    y: np.ndarray,
    regularization: float = 0.0
) -> tuple[np.ndarray, float]:
    """Fit a linear model using least squares.

    Args:
        X: Design matrix of shape (n_samples, n_features).
        y: Target values of shape (n_samples,).
        regularization: L2 regularization strength.

    Returns:
        Tuple of (coefficients, intercept).
    """

Notice how the docstring describes shapes and meanings rather than repeating that X is an ndarray. The type hints handle the type information, and the docstring provides the semantic information.

## Doctest

Doctest lets you embed tests directly in docstrings. Python's `doctest` module extracts and runs these examples, ensuring your documentation stays accurate.

### Writing Doctests

Include examples that look like interactive Python sessions. The `>>>` prefix indicates input, and the following line shows the expected output.

In [ ]:
def factorial(n):
    """Return the factorial of n.

    Args:
        n: Non-negative integer.

    Returns:
        n! (n factorial).

    Examples:
        >>> factorial(0)
        1
        >>> factorial(5)
        120
        >>> factorial(3)
        6
    """
    if n == 0:
        return 1
    return n * factorial(n - 1)

When you run doctest on this module, it will execute `factorial(0)` and verify the output is `1`, then execute `factorial(5)` and verify the output is `120`, and so on.

### Running Doctests

Run doctests from the command line:

In [ ]:
%%bash
python -m doctest mymodule.py

If all tests pass, there's no output. Failed tests show the expected vs actual output.

You can also run doctests from within a script by adding this at the bottom:

In [ ]:
if __name__ == "__main__":
    import doctest
    doctest.testmod()

This runs all doctests in the module when the script is executed directly.

### Benefits for Scientific Code

Doctests are particularly useful for statistical functions because they verify mathematical properties.

In [ ]:
def zscore(x):
    """Standardize values to have mean 0 and standard deviation 1.

    Examples:
        >>> import numpy as np
        >>> data = [1, 2, 3, 4, 5]
        >>> z = zscore(data)
        >>> np.round(np.mean(z), 10)
        0.0
        >>> np.round(np.std(z), 10)
        1.0
    """
    x = np.array(x)
    return (x - np.mean(x)) / np.std(x)

These doctests verify the defining properties of z-scores. If someone modifies the function incorrectly, the doctests will catch it.

### Question

The following doctest is failing. Find the bug in either the function or the doctest.

In [ ]:
def median_absolute_deviation(data):
    """Calculate the median absolute deviation (MAD).

    MAD is a robust measure of spread, defined as the median of
    absolute deviations from the median.

    Examples:
        >>> median_absolute_deviation([1, 2, 3, 4, 5])
        1.0
        >>> median_absolute_deviation([1, 1, 1, 1])
        0.0
        >>> median_absolute_deviation([1, 2, 3, 4, 5, 100])
        1.5
    """
    median = np.median(data)
    return np.median(np.abs(data - median))

Running doctest produces:
```
Failed example:
    median_absolute_deviation([1, 2, 3, 4, 5])
Expected:
    1.0
Got:
    1.0
```

Wait, those look the same! What's happening?

**Answer:**

The issue is floating-point representation. The function returns `np.float64(1.0)`, but the doctest expects Python's `float(1.0)`. These display identically but aren't equal for doctest's comparison.

There are two fixes:

**Fix 1:** Convert to Python float in the doctest:

In [ ]:
>>> float(median_absolute_deviation([1, 2, 3, 4, 5]))
1.0

**Fix 2:** Use `# doctest: +ELLIPSIS` for approximate matching:

In [ ]:
>>> median_absolute_deviation([1, 2, 3, 4, 5])  # doctest: +ELLIPSIS
1.0...

This is a common gotcha when testing NumPy functions with doctest.

## Recommended Resources

* [PEP 257 - Docstring Conventions](https://peps.python.org/pep-0257/) - Official Python docstring standards
* [PEP 484 - Type Hints](https://peps.python.org/pep-0484/) - Type annotation specification
* [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html) - Includes docstring format guidelines
* [Python typing module documentation](https://docs.python.org/3/library/typing.html) - Complete type hints reference
* [Sphinx documentation](https://www.sphinx-doc.org/) - Documentation generator